In [18]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output
import cv2
import numpy as np
from copy import deepcopy

# Set inline plots size
plt.rcParams["figure.figsize"] = (32, 20) # (w, h)

In [19]:
def read_and_resize(filename, grayscale=False, fx=0.5, fy=0.5):
    if grayscale:
      img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
      imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
      img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_result = cv2.resize(img_result, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
    return img_result


def showInRow(list_of_images, titles = None, disable_ticks = False):
  count = len(list_of_images)
  for idx in range(count):
    subplot = plt.subplot(1, count, idx+1)
    if titles is not None:
      subplot.set_title(titles[idx])
      
    img = list_of_images[idx]
    cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
    subplot.imshow(img, cmap=cmap)
    if disable_ticks:
      plt.xticks([]), plt.yticks([])
  plt.show()


def getGaussKernel2d(ksize, sigma):
  kern = cv2.getGaussianKernel(ksize, sigma)
  return kern * np.transpose(kern)

In [20]:
!wget "https://drive.google.com/uc?export=view&id=1FdJqD04_l8n3sFl3QotxOcyH07ndEVm6" -O task3.png
clear_output()

In [21]:
img = read_and_resize("task3.png")
#@title Parameters { run: "auto" }
block_size = 9 #@param {type:"slider", min:1, max:55, step:2}
c_value = 2 #@param {type:"slider", min:-20, max:20, step:1}


def adaptive_thresholding(image, thresholding_type = 'mean' , block_size = 11 , c_value = 5):
  # Resize the image if the block size does not conform with it
  old_size = (image.shape[1], image.shape[0])
  new_size = ((image.shape[1]//block_size)*block_size, (image.shape[0]//block_size)*block_size)
  
  pad = block_size // 2

  # Check if the image in grayscale or not
  if len(image.shape) ==3 :
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Resizing 
  image = cv2.resize(image, dsize = new_size, interpolation = cv2.INTER_CUBIC)
 
  img_cp = image.copy()

  img_bw = np.full(img_cp.shape, 255, dtype=np.uint8)

  # Iterating over the picture
  for i in range(0, img_cp.shape[0], block_size):
        for j in range(0, img_cp.shape[1], block_size):

            hood = img_cp[i:i+block_size, j:j+ block_size]

            # Calculating the threshold depending on the threshold type
            if thresholding_type== 'mean':
              threshold = np.sum(hood)/block_size**2 - c_value
            elif thresholding_type == 'gaussian':
              kernel = getGaussKernel2d(block_size, 3)
              threshold = ((hood * kernel).sum())-c_value

            ret, img_bw_hood = cv2.threshold(hood, threshold, 255, cv2.THRESH_BINARY)
            img_bw[i:i+block_size, j:j+ block_size] = img_bw_hood

  img_bw = cv2.resize(img_bw, dsize = old_size, interpolation = cv2.INTER_CUBIC)
  return img_bw


img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
showing = adaptive_thresholding(img, thresholding_type = 'mean' , block_size = block_size, c_value = c_value)
adap =  cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size, c_value)

showInRow([img, showing, adap], titles=["original", "implemented MEAN", "built-in"])

showing = adaptive_thresholding(img, thresholding_type = 'gaussian', block_size = block_size, c_value = c_value)
adap =  cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c_value)

showInRow([img, showing, adap], titles=["original", "implemented GAUSSIAN", "built-in"])

Output hidden; open in https://colab.research.google.com to view.